In [1]:
from neo4j import __version__ as neo4j_version
print(neo4j_version)

5.1.0


In [2]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
conn = Neo4jConnection(uri="bolt://localhost:11012", user="superman", pwd="pizza")

In [4]:
conn.query("MATCH (n) DETACH DELETE n")

[]

In [5]:
conn.query("""CREATE (matrix1:Movie { title : 'The Matrix', year : '1999-03-31' })
CREATE (matrix2:Movie { title : 'The Matrix Reloaded', year : '2003-05-07' })
CREATE (matrix3:Movie { title : 'The Matrix Revolutions', year : '2003-10-27' })
CREATE (keanu:Actor { name:'Keanu Reeves' })
CREATE (laurence:Actor { name:'Laurence Fishburne' })
CREATE (carrieanne:Actor { name:'Carrie-Anne Moss' })
CREATE (keanu)-[:ACTS_IN { role : 'Neo' }]->(matrix1)
CREATE (keanu)-[:ACTS_IN { role : 'Neo' }]->(matrix2)
CREATE (keanu)-[:ACTS_IN { role : 'Neo' }]->(matrix3)
CREATE (laurence)-[:ACTS_IN { role : 'Morpheus' }]->(matrix1)
CREATE (laurence)-[:ACTS_IN { role : 'Morpheus' }]->(matrix2)
CREATE (laurence)-[:ACTS_IN { role : 'Morpheus' }]->(matrix3)
CREATE (carrieanne)-[:ACTS_IN { role : 'Trinity' }]->(matrix1)
CREATE (carrieanne)-[:ACTS_IN { role : 'Trinity' }]->(matrix2)
CREATE (carrieanne)-[:ACTS_IN { role : 'Trinity' }]->(matrix3)
""")

[]

In [6]:
conn.query("""
CREATE (johnwick:Movie { title : 'John Wick', year : '2014-10-24' })
CREATE (chad:Director { name:'Chad Stahelski' })
CREATE (david:Director { name:'David Leitch' })
CREATE (william:Actor { name:'William Dafoe' })
CREATE (michael:Actor { name:'Michael Nyquist' })
CREATE (william)-[:ACTS_IN { role : 'Marcus' }]->(johnwick)
CREATE (michael)-[:ACTS_IN { role : 'Viggo Tarasov' }]->(johnwick)
CREATE (chad)-[:DIRECTS]->(johnwick)
CREATE (david)-[:DIRECTS]->(johnwick)
""")

[]

In [7]:
conn.query("""MATCH (n {title:'John Wick'}), (m {name:'Keanu Reeves'}) CREATE (m)-[:ACTS_IN { role : 'John Wick' }]->(n)""")

[]

In [8]:
actor_list = conn.query("""
MATCH p = (:Actor {name:'Keanu Reeves'})-[:ACTS_IN]->(:Movie)<-[:ACTS_IN]-(co_actor:Actor) RETURN co_actor
""")

In [9]:
actor_list

[<Record co_actor=<Node element_id='2' labels=frozenset({'Actor'}) properties={'name': 'Michael Nyquist'}>>,
 <Record co_actor=<Node element_id='1' labels=frozenset({'Actor'}) properties={'name': 'William Dafoe'}>>,
 <Record co_actor=<Node element_id='23' labels=frozenset({'Actor'}) properties={'name': 'Carrie-Anne Moss'}>>,
 <Record co_actor=<Node element_id='22' labels=frozenset({'Actor'}) properties={'name': 'Laurence Fishburne'}>>,
 <Record co_actor=<Node element_id='23' labels=frozenset({'Actor'}) properties={'name': 'Carrie-Anne Moss'}>>,
 <Record co_actor=<Node element_id='22' labels=frozenset({'Actor'}) properties={'name': 'Laurence Fishburne'}>>,
 <Record co_actor=<Node element_id='23' labels=frozenset({'Actor'}) properties={'name': 'Carrie-Anne Moss'}>>,
 <Record co_actor=<Node element_id='22' labels=frozenset({'Actor'}) properties={'name': 'Laurence Fishburne'}>>]

In [10]:
co_actorList = []
for actor in actor_list:
    co_actorList.append(actor.data()['co_actor']['name'])

In [11]:
co_actorSet = set(co_actorList)
co_actorSet

{'Carrie-Anne Moss', 'Laurence Fishburne', 'Michael Nyquist', 'William Dafoe'}

In [12]:
director_list = conn.query("""
MATCH p = (:Actor {name:'Keanu Reeves'})-[:ACTS_IN]->(:Movie)<-[:DIRECTS]-(dir:Director) RETURN dir
""")

In [13]:
director_list

[<Record dir=<Node element_id='25' labels=frozenset({'Director'}) properties={'name': 'Chad Stahelski'}>>,
 <Record dir=<Node element_id='0' labels=frozenset({'Director'}) properties={'name': 'David Leitch'}>>]

In [14]:
director_set = []
for director in director_list:
    director_set.append(director.data()['dir']['name'])

In [15]:
director_list = conn.query("""
MATCH p = (:Actor {name:'Keanu Reeves'})-[:ACTS_IN]->(:Movie)<-[:DIRECTS]-(dir:Director) RETURN dir
""")

In [16]:
director_list

[<Record dir=<Node element_id='25' labels=frozenset({'Director'}) properties={'name': 'Chad Stahelski'}>>,
 <Record dir=<Node element_id='0' labels=frozenset({'Director'}) properties={'name': 'David Leitch'}>>]

In [17]:
director_set = set(director_set)
director_set

{'Chad Stahelski', 'David Leitch'}

## Write to CSV, delete existing Graph, then load CSV to Neo4j as Graph

In [18]:
import pandas as pd

# We can only import and export CSV to Neo4j from the below import directory (local)

base_loc = """C:\\Users\\Abhijeet Gupta\\.Neo4jDesktop\\relate-data\\dbmss\\
dbms-8445bcbc-9670-4bf5-aee9-dc7e82460694\\import\\"""

# Get Nodes as DataFrames for each type

movie  = pd.DataFrame(conn.query("MATCH (m:Movie) return m.title as title, m.year as year"),columns=["title","year"])
actor  = pd.DataFrame(conn.query("MATCH (a:Actor) return a.name as name"),columns=["name"])
director  = pd.DataFrame(conn.query("MATCH (a:Director) return a.name as name"),columns=["name"])

# Get Relationships as DataFrames for each type

acts_in = pd.DataFrame(
    conn.query("MATCH (a:Actor)-[r:ACTS_IN]->(m:Movie) return a.name as name, m.title as title, r.role as role"),
    columns=["name","title","role"])

directs_in = pd.DataFrame(
    conn.query("MATCH (d:Director)-[r:DIRECTS]->(m:Movie) return d.name as name, m.title as title"),
    columns=["name","title"])

In [20]:
# Export Nodes/Relationships DataFrames to CSV

movie.to_csv(base_loc + "export_movies.csv")
actor.to_csv(base_loc + "export_actors.csv")
director.to_csv(base_loc + "export_directors.csv")

acts_in.to_csv(base_loc + "export_acts_in.csv")
directs_in.to_csv(base_loc + "export_directs_in.csv")

In [24]:
# delete entire graph

conn.query("MATCH (n) DETACH DELETE n")

[]

In [25]:
# load csv to Neo4j as graph

conn.query("""load csv with headers from 'file:///export_movies.csv' AS
          line CREATE (m:Movie { id:line.id, title:line.title, year:line.year})""")
conn.query("""load csv with headers from 'file:///export_actors.csv' AS
          line CREATE (a:Actor {id:line.id,name:line.name})""")
conn.query("""load csv with headers from 'file:///export_directors.csv' AS
          line CREATE (d:Director { id:line.id,name:line.name})""")

conn.query("""load csv with headers from 'file:///export_acts_in.csv' AS
          line MATCH (m:Movie { title:line.title }) MATCH (a:Actor { name:line.name })
          CREATE (a)-[:ACTS_IN { roles: [line.role]}]->(m)""")
conn.query("""load csv with headers from 'file:///export_directs_in.csv' AS
          line MATCH (m:Movie { title:line.title }) MATCH (d:Director { name:line.name })
          CREATE (d)-[:DIRECTS]->(m)""")

[]